In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols
from statsmodels.api import qqplot, add_constant
import statsmodels.formula.api as smf

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

from sklearn.preprocessing import StandardScaler
from statsmodels.tools.eval_measures import rmse
import matplotlib
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.tree import export_graphviz  
import graphviz

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import plot_tree

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import scale, minmax_scale, robust_scale

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

%matplotlib inline
matplotlib.rc('font', family='AppleGothic')
plt.rc('font', family='AppleGothic')

matplotlib.rc('axes', unicode_minus = False)
from scipy.stats import chi2_contingency
from scipy import stats

from statsmodels.api import Logit
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

import warnings
warnings.filterwarnings('ignore')

# cooking

In [ ]:
cook=pd.read_csv("Cooking.csv",encoding='euc-kr')
cook.rename(columns = {'불량여부':'불량'},inplace=True)
cook.rename(columns = {'발생시간파생':'오류발생시각'},inplace=True)
cook.head()

,순번,생산라인코드,생산라인명,품목코드,품목명,작업장코드,충전실온도,실링온도,쿠킹온도,쿠킹스팀압력,...,생산시간,생산일자,불량,발생시간,오류발생시각,종료시간,오류조치시간,비가동코드,Error_Message,오류발생작업장명
0,3,C001,쿠커 #1,23K01312,서비스면양념장(재),W003,71.669853,96.769853,131.742003,23.315,...,70.37,2021-01-03,양품,0,0,0,0.0,0,0,0
1,1,C002,쿠커 #2,23V00019,고구마샐러드믹스베이스(재),W003,70.386480,102.086480,136.926080,23.226,...,70.83,2021-01-03,양품,0,0,0,0.0,0,0,0
2,13,C003,쿠커 #3,1.#JE+00,이마트가쓰오다시국물(재),W003,71.206130,112.306130,147.579680,23.516,...,73.55,2021-01-03,양품,0,0,0,0.0,0,0,0
3,7,C005,2동 쿠커 #4,26A01006,고구마샐러드바145g(재),W003,70.430565,96.030565,131.569165,23.692,...,74.81,2021-01-03,양품,0,0,0,0.0,0,0,0
4,2,C007,프리머 #1,26B06024,이츠웰골드마요네즈1kg(재),W003,72.179318,114.079318,149.928068,23.899,...,73.40,2021-01-03,양품,0,0,0,0.0,0,0,0


## 제품군 범주화

In [ ]:
def func1(row):
    if pd.isnull(row):
        return row
    
    if '소스' in row:
        return 'source'
    elif '드레싱' in row :
        return 'source'
    elif '마요' in row :
        return 'source'
    elif '밥' in row :
        return 'rice'
    elif '비빔밥' in row :
        return 'rice'
    elif '볶음밥' in row :
        return 'rice'
    elif '도시락' in row:
        return '도시락'
    elif '샐러드' in row:
        return 'salad'
    elif '베이스' in row:
        return '육수베이스'
    elif '국물' in row:
        return '육수베이스'
    elif '양념' in row:
        return 'source'
    elif '허브' in row:
        return 'spice'
    elif '디핑' in row:
        return 'source'  
    elif '야채케ㅤㅊㅑㅂ' in row:
        return 'source'  
        
    
cook['구분'] = cook['품목명'].apply(func1)

## 제품군 불량개수 및 비율

In [ ]:
cook['불량'] = cook['불량'].replace('불량', 1).replace('양품', 0)

len_p1=cook.groupby('구분')['구분'].count()
len_p1=pd.DataFrame(len_p1)
len_p1.rename(columns={'구분':'개수'},inplace=True)
len_p1.reset_index(inplace=True)
len_p1

p1 = cook.pivot_table(index='구분' , values= '불량', aggfunc='sum').reset_index()
p1['비율'] = p1['불량'] / len_p1['개수']
p1

## salad 항목이 불량 비율이 가장 높다.


,구분,불량,비율
0,rice,428,0.044930
1,salad,53,0.048579
2,source,475,0.043981
3,spice,1,0.031250
4,육수베이스,30,0.042796


## 비율전체불량개수 (전체불량개수 중 각 제품군별 불량률)

In [ ]:
sum_all=sum(p1['불량'])
sum_all

p1['비율_전체불량개수']=p1['불량']/sum_all

In [ ]:
len_p1 

In [ ]:
cook.to_csv("Cooking.csv",index= False ,encoding= 'euc-kr')

# Booking

In [ ]:
book=pd.read_csv("Booking.csv")

In [ ]:
book=book.iloc[:,1:]
book.rename(columns = {'수주품목코드':'품목코드'},inplace=True)

In [ ]:
book_merge_cook=book.merge(cook, on='품목코드',how='inner')

In [ ]:
def func1(row):
    if pd.isnull(row):
        return 'None'
    if '소스' in row:
        return 'source'
    elif '드레싱' in row :
        return 'source'
    elif '마요' in row :
        return 'source'
    elif '밥' in row :
        return 'rice'
    elif '비빔밥' in row :
        return 'rice'
    elif '볶음밥' in row :
        return 'rice'
    elif '도시락' in row:
        return '도시락'
    elif '샐러드' in row:
        return 'salad'
    elif '베이스' in row:
        return '육수베이스'
    elif '국물' in row:
        return '육수베이스'
    elif '양념' in row:
        return 'source'
    elif '허브' in row:
        return 'spice'
    elif '디핑' in row:
        return 'source'  
    elif '야채케ㅤㅊㅑㅂ' in row:
        return 'source'  
        
    
    
book_merge_cook['구분'] = book_merge_cook['품목명'].apply(func1)

In [ ]:
book_merge_cook.구분.unique()

array(['salad', 'rice', 'source', None, '육수베이스', 'spice'], dtype=object)

In [ ]:
book_merge_cook.columns

Index(['수주사업장', '수주일자', '수주일련번호', '거래처코드', '수주부서', '품목코드', '출고수량KG', '수주수량KG',
       '수주단가', '수주금액', '부가세금액', '출하완료여부', '납기일자', '순번', '생산라인코드', '생산라인명',
       '품목명', '작업장코드', '충전실온도', '실링온도', '쿠킹온도', '쿠킹스팀압력', '실링압력', '생산시간',
       '생산일자', '불량', '발생시간', '오류발생시각', '종료시간', '오류조치시간', '비가동코드',
       'Error_Message', '오류발생작업장명', '구분'],
      dtype='object')

## 출하완료여부 N - 1 , Y - 0 변환

In [ ]:
book_merge_cook['출하완료여부'] = book_merge_cook['출하완료여부'].replace('N', 1).replace('Y', 0)

## 범주화 구분

In [ ]:
len_p1=book_merge_cook.groupby('구분')['구분'].count()
len_p1=pd.DataFrame(len_p1)
len_p1.rename(columns={'구분':'개수'},inplace=True)
len_p1.reset_index(inplace=True)
len_p1

,구분,개수
0,rice,2977699
1,salad,2746211
2,source,11704537
3,spice,106795
4,육수베이스,16057


## 출화완료 여부 개수 및 비율

In [ ]:
p1 = book_merge_cook.pivot_table(index='구분' , values= '출하완료여부', aggfunc='sum').reset_index()
p1['비율'] = p1['출하완료여부'] / len_p1['개수']
p1 #rice 품목의 납기미준수비율이 가장 높다!

,구분,출하완료여부,비율
0,rice,8948,0.003005
1,salad,6700,0.002440
2,source,19522,0.001668
3,spice,0,0.000000
4,육수베이스,37,0.002304


In [ ]:
sum_all = sum(p1['출하완료여부'])
p1['비율_전체출하실패개수'] = p1['출하완료여부'] / sum_all
p1 #source 가 가장 출하실패비율을 많이 차지!

,구분,출하완료여부,비율,비율_전체출하실패개수
0,rice,8948,0.003005,0.254154
1,salad,6700,0.002440,0.190303
2,source,19522,0.001668,0.554492
3,spice,0,0.000000,0.000000
4,육수베이스,37,0.002304,0.001051
